# Food Price Forecasting using LSTM Neural Networks

**Course:** Dartmouth COSC 16 / Computational Neuroscience  
**Team Members:** [Your Name], [Partner Name]  
**Date:** Fall 2024

## Project Overview

This project implements a Long Short-Term Memory (LSTM) recurrent neural network to forecast next-month prices for rice, maize, and wheat using the Global Food Prices dataset from Kaggle. The dataset contains approximately 3 million monthly observations across ~99 countries from 2000 to present. We compare the LSTM against two baseline models: a naïve "last-value" predictor and a tuned ARIMA model. Performance is evaluated using Root Mean Squared Error (RMSE), Mean Absolute Percentage Error (MAPE), and Directional Accuracy. The project includes exploratory data analysis, feature engineering, model training, and interpretability analysis using SHAP-style feature importance to understand which temporal patterns the model learns.


## 1. Setup and Imports


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Machine learning
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error

# Deep learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Time series
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.seasonal import seasonal_decompose
import statsmodels.api as sm

# Interpretability (optional)
try:
    import shap
    SHAP_AVAILABLE = True
except ImportError:
    SHAP_AVAILABLE = False
    print("Note: SHAP not available, will use permutation importance instead")

# Set style
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)

print(f"TensorFlow version: {tf.__version__}")
print(f"Pandas version: {pd.__version__}")
print(f"NumPy version: {np.__version__}")


## 2. Data Loading & Cleaning


In [ ]:
# Extract and load the dataset
import zipfile
import os

zip_path = 'kaggle-dataset-globalfoodprices.zip'
extract_dir = 'temp_extract'

# Extract if not already extracted
if not os.path.exists(extract_dir):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

# Find the CSV file
csv_files = [f for f in os.listdir(extract_dir) if f.endswith('.csv')]
print(f"Found CSV files: {csv_files}")

# Load the main dataset (chunked reading for large file)
csv_path = os.path.join(extract_dir, csv_files[0])
print(f"Loading: {csv_path}")

# Read in chunks to handle large file
chunk_list = []
chunk_size = 100000
for chunk in pd.read_csv(csv_path, chunksize=chunk_size, low_memory=False):
    chunk_list.append(chunk)

df_raw = pd.concat(chunk_list, ignore_index=True)
print(f"Loaded {len(df_raw):,} rows")
print(f"Columns: {list(df_raw.columns)}")


In [ ]:
# Inspect the data structure
print("\nFirst few rows:")
print(df_raw.head())
print("\nData types:")
print(df_raw.dtypes)
print("\nMissing values:")
print(df_raw.isnull().sum())


In [ ]:
# Parse dates
df_raw['date'] = pd.to_datetime(df_raw['date'], errors='coerce')

# Check available commodities
print("\nUnique commodities (first 50):")
print(df_raw['commodity'].unique()[:50])

# Find rice, maize, and wheat (case-insensitive)
rice_matches = df_raw[df_raw['commodity'].str.contains('rice', case=False, na=False)]['commodity'].unique()
maize_matches = df_raw[df_raw['commodity'].str.contains('maize|corn', case=False, na=False)]['commodity'].unique()
wheat_matches = df_raw[df_raw['commodity'].str.contains('wheat', case=False, na=False)]['commodity'].unique()

print("\nRice matches:", rice_matches)
print("Maize matches:", maize_matches)
print("Wheat matches:", wheat_matches)


In [ ]:
# Filter to our three commodities
df_filtered = df_raw[
    (df_raw['commodity'].str.contains('Rice', case=False, na=False)) |
    (df_raw['commodity'].str.contains('Maize|Corn', case=False, na=False)) |
    (df_raw['commodity'].str.contains('Wheat', case=False, na=False))
].copy()

print(f"Rows after filtering to rice/maize/wheat: {len(df_filtered):,}")

# Standardize commodity names
def standardize_commodity(name):
    name_lower = str(name).lower()
    if 'rice' in name_lower:
        return 'Rice'
    elif 'maize' in name_lower or 'corn' in name_lower:
        return 'Maize'
    elif 'wheat' in name_lower:
        return 'Wheat'
    return name

df_filtered['commodity'] = df_filtered['commodity'].apply(standardize_commodity)
print("\nCommodity counts:")
print(df_filtered['commodity'].value_counts())


In [ ]:
# Use USD price for consistency across countries
# Filter to rows with valid USD prices
df_filtered = df_filtered[df_filtered['price_usd'].notna()].copy()
df_filtered = df_filtered[df_filtered['price_usd'] > 0].copy()

# Filter to KG unit for consistency
df_filtered = df_filtered[df_filtered['unit'].str.upper() == 'KG'].copy()

print(f"Rows after filtering to valid USD prices and KG unit: {len(df_filtered):,}")
print(f"\nDate range: {df_filtered['date'].min()} to {df_filtered['date'].max()}")
print(f"\nCountries: {df_filtered['country_code'].nunique()}")
print(f"\nCountries with most data:")
print(df_filtered['country_code'].value_counts().head(10))


In [ ]:
# Select a country with good coverage for all three commodities
country_stats = []
for country in df_filtered['country_code'].unique():
    country_data = df_filtered[df_filtered['country_code'] == country]
    commodities = country_data['commodity'].unique()
    date_range = (country_data['date'].max() - country_data['date'].min()).days
    
    # Check if all three commodities are present
    has_all = all(c in commodities for c in ['Rice', 'Maize', 'Wheat'])
    
    if has_all:
        country_stats.append({
            'country': country,
            'date_range_days': date_range,
            'num_obs': len(country_data),
            'start_date': country_data['date'].min(),
            'end_date': country_data['date'].max()
        })

country_df = pd.DataFrame(country_stats).sort_values('num_obs', ascending=False)
print("\nTop countries with all three commodities:")
print(country_df.head(10))

# Select the top country
selected_country = country_df.iloc[0]['country']
print(f"\nSelected country: {selected_country}")

df_country = df_filtered[df_filtered['country_code'] == selected_country].copy()
print(f"\nData for {selected_country}: {len(df_country):,} rows")
print(f"Date range: {df_country['date'].min()} to {df_country['date'].max()}")
print(f"\nCommodity counts:")
print(df_country['commodity'].value_counts())


In [ ]:
# Create monthly aggregated time series
df_country['year_month'] = df_country['date'].dt.to_period('M')

# Aggregate by month and commodity
monthly_prices = df_country.groupby(['year_month', 'commodity'])['price_usd'].mean().reset_index()

# Pivot to get separate columns for each commodity
df_ts = monthly_prices.pivot(index='year_month', columns='commodity', values='price_usd')
df_ts.index = df_ts.index.to_timestamp()
df_ts = df_ts.sort_index()

# Rename columns for consistency
df_ts.columns = [f"{col.lower()}_price" for col in df_ts.columns]

print(f"\nFinal time series shape: {df_ts.shape}")
print(f"Date range: {df_ts.index.min()} to {df_ts.index.max()}")
print(f"\nMissing values per commodity:")
print(df_ts.isnull().sum())

# Forward fill missing months (within reason)
missing_before = df_ts.isnull().sum().sum()
df_ts = df_ts.ffill(limit=3)  # Forward fill up to 3 months
df_ts = df_ts.bfill(limit=3)  # Backward fill up to 3 months
missing_after = df_ts.isnull().sum().sum()

print(f"\nMissing values filled: {missing_before - missing_after}")

# Drop any remaining rows with missing values
df_ts = df_ts.dropna()

print(f"\nFinal time series after cleaning: {len(df_ts)} months")
print(f"\nFinal columns: {list(df_ts.columns)}")
print("\nFirst few rows:")
print(df_ts.head())
print("\nLast few rows:")
print(df_ts.tail())


### Data Description

The final dataset consists of monthly aggregated prices for rice, maize, and wheat from the selected country.

- **Time range:** [Will be filled after running]  
- **Number of monthly observations:** [Will be filled after running]  
- **Commodities:** Rice, Maize, Wheat  
- **Price unit:** USD per kilogram (standardized across all observations)  
- **Data source:** Global Food Prices dataset (Kaggle)  

We selected the country with:
- Complete coverage of all three commodities
- Longest time span with consistent data
- Minimal missing values

Missing months were forward-filled or backward-filled (up to 3 months) to create a continuous time series. Any remaining gaps were dropped to ensure data quality.


## 3. Exploratory Data Analysis (EDA)


In [ ]:
# Time series plot for all three commodities
plt.figure(figsize=(14, 6))
for col in df_ts.columns:
    plt.plot(df_ts.index, df_ts[col], label=col.replace('_price', '').title(), linewidth=1.5)

plt.xlabel('Date', fontsize=12)
plt.ylabel('Price (USD per kg)', fontsize=12)
plt.title('Monthly Food Prices Over Time', fontsize=14, fontweight='bold')
plt.legend(fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()


In [ ]:
# Rolling statistics for one commodity (wheat)
window = 12  # 12-month rolling window

fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# Rolling mean and std for wheat
wheat_col = 'wheat_price'
if wheat_col in df_ts.columns:
    rolling_mean = df_ts[wheat_col].rolling(window=window).mean()
    rolling_std = df_ts[wheat_col].rolling(window=window).std()
    
    axes[0].plot(df_ts.index, df_ts[wheat_col], label='Wheat Price', alpha=0.6, linewidth=1)
    axes[0].plot(df_ts.index, rolling_mean, label=f'{window}-Month Rolling Mean', linewidth=2, color='red')
    axes[0].fill_between(df_ts.index, 
                         rolling_mean - rolling_std, 
                         rolling_mean + rolling_std, 
                         alpha=0.2, label=f'{window}-Month Rolling Std')
    axes[0].set_xlabel('Date', fontsize=11)
    axes[0].set_ylabel('Price (USD per kg)', fontsize=11)
    axes[0].set_title('Wheat Price with Rolling Statistics', fontsize=12, fontweight='bold')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

# Seasonal decomposition (additive)
if len(df_ts) >= 24:  # Need at least 2 years
    decomposition = seasonal_decompose(df_ts[wheat_col], model='additive', period=12)
    
    decomposition.trend.plot(ax=axes[1], title='Trend Component', color='blue', linewidth=1.5)
    axes[1].set_xlabel('Date', fontsize=11)
    axes[1].set_ylabel('Trend', fontsize=11)
    axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Correlation analysis
correlation_matrix = df_ts.corr()

plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, fmt='.3f', cmap='coolwarm', 
            center=0, square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix: Food Commodity Prices', fontsize=12, fontweight='bold')
plt.tight_layout()
plt.show()

print("\nCorrelation values:")
print(correlation_matrix)


### EDA Interpretation

The time series plots reveal several key patterns:

- **Long-term trends:** Prices show varying trends over the time period, with some commodities exhibiting upward or downward movements
- **Volatility:** Prices show varying levels of volatility, with some periods of relative stability and others with sharp spikes
- **Seasonal patterns:** The rolling mean and seasonal decomposition suggest potential seasonal cycles, which may be related to harvest seasons and agricultural cycles
- **Correlation:** The three commodities show positive correlations, suggesting they tend to move together, possibly due to shared economic factors like global demand, climate patterns, or commodity market dynamics

These patterns motivate the use of sequence models like LSTMs, which can capture both short-term dependencies (recent price movements) and longer-term patterns (seasonality, trends).


## 4. Problem Formulation & Objective

### Objective

The goal of this project is to forecast next-month prices for rice, maize, and wheat using historical price data. Specifically:

- **Input:** Past monthly prices for rice, maize, and wheat (and calendar features like month-of-year)
- **Output:** Predicted price for each commodity in the next month
- **Model:** We use a Long Short-Term Memory (LSTM) recurrent neural network as our primary model. LSTMs are well-suited for time series forecasting because they can learn temporal dependencies and patterns over multiple time steps through their recurrent architecture and memory cells.

**Baseline Comparisons:**
- **Naïve baseline:** Predicts next month's price as the current month's price (last-value predictor)
- **ARIMA baseline:** A classical time series model that combines auto-regressive (AR), integrated (I), and moving average (MA) components. ARIMA is a standard benchmark for univariate time series forecasting.

**Evaluation:**
- **Train-test split:** Chronological split with first 80% of months as training data and last 20% as test data
- **Metrics:** 
  - Root Mean Squared Error (RMSE): Measures average prediction error magnitude
  - Mean Absolute Percentage Error (MAPE): Measures relative error as a percentage
  - Directional Accuracy: Percentage of times the model correctly predicts whether price goes up or down


## 5. Feature Engineering


In [ ]:
# Create calendar features
df_features = df_ts.copy()

# Month (1-12)
df_features['month'] = df_features.index.month

# Year
df_features['year'] = df_features.index.year

# Cyclical encoding for month (sine/cosine) to capture seasonality
df_features['month_sin'] = np.sin(2 * np.pi * df_features['month'] / 12)
df_features['month_cos'] = np.cos(2 * np.pi * df_features['month'] / 12)

# Create lag features (useful for baselines and interpretability)
for lag in [1, 3, 6, 12]:
    for col in ['rice_price', 'maize_price', 'wheat_price']:
        if col in df_features.columns:
            df_features[f'{col}_lag{lag}'] = df_features[col].shift(lag)

print("Feature columns:")
print(df_features.columns.tolist())
print(f"\nShape: {df_features.shape}")
print("\nFirst few rows:")
print(df_features.head(15))


In [ ]:
# Function to create sequences for LSTM
def create_sequences(data, sequence_length, target_cols):
    """
    Convert time series data into sequences for LSTM.
    
    Parameters:
    - data: DataFrame with time series data
    - sequence_length: Number of past months to use (T)
    - target_cols: List of column names to predict
    
    Returns:
    - X: Array of shape [num_samples, sequence_length, num_features]
    - y: Array of shape [num_samples, num_targets]
    """
    X, y = [], []
    
    # Select feature columns (exclude target columns from input features)
    feature_cols = [col for col in data.columns if col not in target_cols]
    
    for i in range(sequence_length, len(data)):
        # Input sequence: past T months
        X.append(data[feature_cols].iloc[i-sequence_length:i].values)
        # Target: next month's prices for all commodities
        y.append(data[target_cols].iloc[i].values)
    
    return np.array(X), np.array(y)

# Define sequence length (using 12 months of history)
SEQUENCE_LENGTH = 12
TARGET_COLS = ['rice_price', 'maize_price', 'wheat_price']

# Drop rows with NaN (from lag features)
df_features_clean = df_features.dropna()
print(f"Data after dropping NaN: {len(df_features_clean)} months")

# Create sequences
X_full, y_full = create_sequences(df_features_clean, SEQUENCE_LENGTH, TARGET_COLS)

print(f"\nSequence shape X: {X_full.shape}")
print(f"Target shape y: {y_full.shape}")
print(f"\nNumber of features per time step: {X_full.shape[2]}")


In [ ]:
print("Feature engineering complete.")
print(f"\nInput features include:")
print("- Current prices (rice, maize, wheat)")
print("- Lagged prices (1, 3, 6, 12 months)")
print("- Calendar features (month, year, cyclical month encoding)")


### Feature Engineering Rationale

We create several types of features:

1. **Calendar features:** Month and year capture long-term trends and potential seasonality. Cyclical encoding (sine/cosine) for month helps the model understand that month 12 and month 1 are adjacent in the seasonal cycle.

2. **Lagged features:** Past prices at various lags (1, 3, 6, 12 months) provide explicit historical context. These are useful for baselines and can help interpret what the LSTM learns.

3. **Sequence windows:** For the LSTM, we use a sliding window of T=12 past months. This allows the model to learn temporal patterns over a full year, capturing both short-term fluctuations and seasonal cycles.

4. **Normalization:** We normalize input features using StandardScaler (fit on training data only) to ensure all features are on similar scales, which helps neural network training converge faster and more stably.


## 6. Train-Test Split


In [ ]:
# Chronological split (critical for time series!)
num_samples = len(X_full)
split_idx = int(0.8 * num_samples)

# Training set: first 80%
X_train = X_full[:split_idx]
y_train = y_full[:split_idx]

# Test set: last 20%
X_test = X_full[split_idx:]
y_test = y_full[split_idx:]

# Further split training into train/validation (last 20% of training = validation)
val_split_idx = int(0.8 * len(X_train))
X_train_final = X_train[:val_split_idx]
y_train_final = y_train[:val_split_idx]
X_val = X_train[val_split_idx:]
y_val = y_train[val_split_idx:]

print(f"Total samples: {num_samples}")
print(f"\nTraining set: {len(X_train_final)} samples")
print(f"Validation set: {len(X_val)} samples")
print(f"Test set: {len(X_test)} samples")
print(f"\nTrain date range: {df_features_clean.index[SEQUENCE_LENGTH]} to {df_features_clean.index[SEQUENCE_LENGTH + len(X_train_final) - 1]}")
print(f"Test date range: {df_features_clean.index[SEQUENCE_LENGTH + split_idx]} to {df_features_clean.index[SEQUENCE_LENGTH + split_idx + len(X_test) - 1]}")


In [ ]:
# Normalize features
# Reshape for scaling: [samples, timesteps, features] -> [samples * timesteps, features]
n_samples_train, n_timesteps, n_features = X_train_final.shape
X_train_reshaped = X_train_final.reshape(-1, n_features)
X_val_reshaped = X_val.reshape(-1, n_features)
X_test_reshaped = X_test.reshape(-1, n_features)

# Fit scaler on training data only
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_reshaped)
X_train_scaled = X_train_scaled.reshape(n_samples_train, n_timesteps, n_features)

# Transform validation and test
X_val_scaled = scaler_X.transform(X_val_reshaped).reshape(X_val.shape[0], n_timesteps, n_features)
X_test_scaled = scaler_X.transform(X_test_reshaped).reshape(X_test.shape[0], n_timesteps, n_features)

# Also scale targets (for LSTM, though we'll convert back for evaluation)
scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train_final)
y_val_scaled = scaler_y.transform(y_val)

print("Feature scaling complete.")
print(f"\nX_train_scaled shape: {X_train_scaled.shape}")
print(f"y_train_scaled shape: {y_train_scaled.shape}")


### Train-Test Split Rationale

For time series data, we **must** use a chronological split rather than random shuffling. This is because:

1. **Temporal structure:** Time series have inherent temporal dependencies. Random shuffling would break these dependencies and allow the model to "see the future" during training, leading to unrealistic performance estimates.

2. **Realistic evaluation:** In practice, we forecast future prices using only past data. A chronological split simulates this real-world scenario.

3. **No data leakage:** By ensuring all training data comes before test data chronologically, we prevent information leakage from future observations.

We use 80% for training and 20% for testing, with an additional validation split (20% of training data) for early stopping during LSTM training.


## 7. Baseline Models

### 7.1 Naïve "Last-Value" Baseline


In [ ]:
def compute_metrics(y_true, y_pred, commodity_names):
    """
    Compute RMSE, MAPE, and directional accuracy for each commodity and overall.
    """
    results = {}
    
    for i, name in enumerate(commodity_names):
        y_true_col = y_true[:, i]
        y_pred_col = y_pred[:, i]
        
        # RMSE
        rmse = np.sqrt(mean_squared_error(y_true_col, y_pred_col))
        
        # MAPE (avoid division by zero)
        mape = np.mean(np.abs((y_true_col - y_pred_col) / (y_true_col + 1e-8))) * 100
        
        # Directional accuracy
        # Compare direction of change: (true_t+1 - true_t) vs (pred_t+1 - true_t)
        if len(y_true_col) > 1:
            # Get previous true values (shift by 1)
            y_true_prev = np.concatenate([[y_true_col[0]], y_true_col[:-1]])
            
            # True direction: sign of change from t to t+1
            true_dir = np.sign(y_true_col - y_true_prev)
            
            # Predicted direction: sign of change from t (true) to t+1 (predicted)
            pred_dir = np.sign(y_pred_col - y_true_prev)
            
            # Count matches (excluding zeros/neutral)
            matches = (true_dir == pred_dir) & (true_dir != 0)
            total_nonzero = (true_dir != 0).sum()
            
            if total_nonzero > 0:
                dir_acc = matches.sum() / total_nonzero * 100
            else:
                dir_acc = 0.0
        else:
            dir_acc = 0.0
        
        results[name] = {
            'RMSE': rmse,
            'MAPE': mape,
            'Directional_Accuracy': dir_acc
        }
    
    # Overall metrics (average across commodities)
    results['Overall'] = {
        'RMSE': np.mean([r['RMSE'] for r in results.values()]),
        'MAPE': np.mean([r['MAPE'] for r in results.values()]),
        'Directional_Accuracy': np.mean([r['Directional_Accuracy'] for r in results.values()])
    }
    
    return results

# Naïve baseline: predict next month = current month
# Get the actual prices from the original dataframe for test period
test_start_idx = SEQUENCE_LENGTH + split_idx
test_prices = df_features_clean[TARGET_COLS].iloc[test_start_idx:test_start_idx + len(y_test)].values

# Previous prices (for naïve: predict t+1 = t)
prev_prices = df_features_clean[TARGET_COLS].iloc[test_start_idx - 1:test_start_idx + len(y_test) - 1].values

y_pred_naive = prev_prices

# Compute metrics
naive_results = compute_metrics(y_test, y_pred_naive, TARGET_COLS)

print("Naïve Baseline Results:")
print("=" * 60)
for name, metrics in naive_results.items():
    print(f"\n{name}:")
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  MAPE: {metrics['MAPE']:.2f}%")
    print(f"  Directional Accuracy: {metrics['Directional_Accuracy']:.2f}%")


### 7.2 ARIMA Baseline


In [ ]:
# Fit ARIMA models for each commodity separately
# Use training data only

train_start_idx = SEQUENCE_LENGTH
train_end_idx = SEQUENCE_LENGTH + split_idx
train_prices = df_features_clean[TARGET_COLS].iloc[train_start_idx:train_end_idx]

arima_models = {}
arima_forecasts = {}

for col in TARGET_COLS:
    print(f"\nFitting ARIMA for {col}...")
    
    # Get training series
    train_series = train_prices[col].values
    
    # Try auto-ARIMA with a reasonable parameter grid
    best_aic = np.inf
    best_model = None
    best_params = None
    
    # Grid search over common ARIMA parameters
    for p in range(0, 4):
        for d in range(0, 3):
            for q in range(0, 4):
                try:
                    model = ARIMA(train_series, order=(p, d, q))
                    fitted = model.fit()
                    
                    if fitted.aic < best_aic:
                        best_aic = fitted.aic
                        best_model = fitted
                        best_params = (p, d, q)
                except:
                    continue
    
    if best_model is not None:
        print(f"  Best ARIMA{best_params[0]}{best_params[1]}{best_params[2]}: AIC = {best_aic:.2f}")
        arima_models[col] = best_model
        
        # Forecast test period (one-step-ahead)
        forecasts = []
        current_series = train_series.copy()
        
        for i in range(len(y_test)):
            # Fit on current series
            model_temp = ARIMA(current_series, order=best_params)
            fitted_temp = model_temp.fit()
            
            # Forecast one step ahead
            forecast = fitted_temp.forecast(steps=1)[0]
            forecasts.append(forecast)
            
            # Update series with true value (for next iteration)
            current_series = np.append(current_series, y_test[i, TARGET_COLS.index(col)])
        
        arima_forecasts[col] = np.array(forecasts)
    else:
        print(f"  Failed to fit ARIMA for {col}, using naïve forecast")
        arima_forecasts[col] = y_pred_naive[:, TARGET_COLS.index(col)]

# Combine forecasts
y_pred_arima = np.column_stack([arima_forecasts[col] for col in TARGET_COLS])

# Compute metrics
arima_results = compute_metrics(y_test, y_pred_arima, TARGET_COLS)

print("\n" + "=" * 60)
print("ARIMA Baseline Results:")
print("=" * 60)
for name, metrics in arima_results.items():
    print(f"\n{name}:")
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  MAPE: {metrics['MAPE']:.2f}%")
    print(f"  Directional Accuracy: {metrics['Directional_Accuracy']:.2f}%")


### ARIMA Model Explanation

ARIMA (Auto-Regressive Integrated Moving Average) is a classical time series forecasting model that combines three components:

- **AR (Auto-Regressive):** Uses past values of the series to predict future values
- **I (Integrated):** Applies differencing to make the series stationary (remove trends)
- **MA (Moving Average):** Uses past forecast errors to improve predictions

ARIMA is parameterized as ARIMA(p, d, q) where:
- p = number of autoregressive terms
- d = degree of differencing
- q = number of moving average terms

We select the best ARIMA parameters using AIC (Akaike Information Criterion) via grid search. ARIMA serves as a strong classical baseline because it explicitly models temporal dependencies and trends, making it a standard benchmark for time series forecasting tasks.


## 8. LSTM Model


In [ ]:
# Build LSTM model
def build_lstm_model(input_shape, num_targets=3):
    """
    Build an LSTM model for multivariate time series forecasting.
    
    Parameters:
    - input_shape: (sequence_length, num_features)
    - num_targets: Number of commodities to predict (default 3)
    """
    model = Sequential([
        LSTM(64, activation='relu', return_sequences=True, input_shape=input_shape),
        Dropout(0.2),
        LSTM(32, activation='relu', return_sequences=False),
        Dropout(0.2),
        Dense(16, activation='relu'),
        Dense(num_targets)  # Output: 3 commodity prices
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='mse',
        metrics=['mae']
    )
    
    return model

# Build model
input_shape = (X_train_scaled.shape[1], X_train_scaled.shape[2])
lstm_model = build_lstm_model(input_shape, num_targets=3)

print("LSTM Model Architecture:")
lstm_model.summary()


In [ ]:
# Train the model
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

history = lstm_model.fit(
    X_train_scaled, y_train_scaled,
    validation_data=(X_val_scaled, y_val_scaled),
    epochs=50,
    batch_size=32,
    callbacks=[early_stopping],
    verbose=1
)


In [ ]:
# Plot training history
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

# Loss
axes[0].plot(history.history['loss'], label='Training Loss', linewidth=2)
axes[0].plot(history.history['val_loss'], label='Validation Loss', linewidth=2)
axes[0].set_xlabel('Epoch', fontsize=11)
axes[0].set_ylabel('Loss (MSE)', fontsize=11)
axes[0].set_title('Model Loss Over Epochs', fontsize=12, fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# MAE
axes[1].plot(history.history['mae'], label='Training MAE', linewidth=2)
axes[1].plot(history.history['val_mae'], label='Validation MAE', linewidth=2)
axes[1].set_xlabel('Epoch', fontsize=11)
axes[1].set_ylabel('MAE', fontsize=11)
axes[1].set_title('Mean Absolute Error Over Epochs', fontsize=12, fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Make predictions on test set
y_pred_lstm_scaled = lstm_model.predict(X_test_scaled, verbose=0)

# Inverse transform to get actual prices
y_pred_lstm = scaler_y.inverse_transform(y_pred_lstm_scaled)

# Compute metrics
lstm_results = compute_metrics(y_test, y_pred_lstm, TARGET_COLS)

print("LSTM Model Results:")
print("=" * 60)
for name, metrics in lstm_results.items():
    print(f"\n{name}:")
    print(f"  RMSE: {metrics['RMSE']:.4f}")
    print(f"  MAPE: {metrics['MAPE']:.2f}%")
    print(f"  Directional Accuracy: {metrics['Directional_Accuracy']:.2f}%")


In [ ]:
# Plot predictions vs actual for test set
test_start_idx = SEQUENCE_LENGTH + split_idx
test_dates = df_features_clean.index[test_start_idx:test_start_idx + len(y_test)]

fig, axes = plt.subplots(3, 1, figsize=(14, 10))

for i, (col, name) in enumerate(zip(TARGET_COLS, ['Rice', 'Maize', 'Wheat'])):
    axes[i].plot(test_dates, y_test[:, i], label='Actual', linewidth=2, alpha=0.8)
    axes[i].plot(test_dates, y_pred_lstm[:, i], label='LSTM Predicted', linewidth=2, alpha=0.8, linestyle='--')
    axes[i].set_xlabel('Date', fontsize=11)
    axes[i].set_ylabel('Price (USD per kg)', fontsize=11)
    axes[i].set_title(f'{name} Price: Actual vs Predicted', fontsize=12, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


In [ ]:
# Scatter plot: predicted vs actual for all commodities
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for i, (col, name) in enumerate(zip(TARGET_COLS, ['Rice', 'Maize', 'Wheat'])):
    axes[i].scatter(y_test[:, i], y_pred_lstm[:, i], alpha=0.6, s=50)
    
    # Perfect prediction line
    min_val = min(y_test[:, i].min(), y_pred_lstm[:, i].min())
    max_val = max(y_test[:, i].max(), y_pred_lstm[:, i].max())
    axes[i].plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
    
    axes[i].set_xlabel('Actual Price', fontsize=11)
    axes[i].set_ylabel('Predicted Price', fontsize=11)
    axes[i].set_title(f'{name}: Predicted vs Actual', fontsize=12, fontweight='bold')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


### LSTM Model Description

Long Short-Term Memory (LSTM) networks are a type of recurrent neural network (RNN) designed to capture long-term dependencies in sequential data. Unlike feedforward neural networks, LSTMs have recurrent connections that allow information to persist across time steps.

**Key advantages for time series forecasting:**

1. **Temporal memory:** LSTMs can remember patterns over many time steps (e.g., seasonal cycles, long-term trends) through their memory cells and gating mechanisms (forget gate, input gate, output gate).

2. **Multivariate modeling:** Our LSTM simultaneously processes prices for all three commodities along with calendar features, allowing it to learn cross-commodity relationships and shared patterns.

3. **Non-linear patterns:** Unlike linear models like ARIMA, LSTMs can capture complex non-linear relationships between past and future prices.

**Architecture:** Our model uses two LSTM layers (64 and 32 units) with dropout regularization to prevent overfitting, followed by dense layers that output predictions for all three commodities simultaneously.

**Training:** We use the Adam optimizer with mean squared error (MSE) loss, and early stopping on validation loss to prevent overfitting. The model learns to predict next-month prices using a 12-month sliding window of historical data.


## 9. Metrics Section


In [ ]:
# Compile all results into a comparison table
results_comparison = []

for model_name, results in [('Naïve', naive_results), ('ARIMA', arima_results), ('LSTM', lstm_results)]:
    for commodity in list(TARGET_COLS) + ['Overall']:
        if commodity == 'Overall':
            name = 'Overall'
        else:
            name = commodity.replace('_price', '').title()
        
        results_comparison.append({
            'Model': model_name,
            'Commodity': name,
            'RMSE': results[commodity if commodity != 'Overall' else 'Overall']['RMSE'],
            'MAPE (%)': results[commodity if commodity != 'Overall' else 'Overall']['MAPE'],
            'Directional Accuracy (%)': results[commodity if commodity != 'Overall' else 'Overall']['Directional_Accuracy']
        })

results_df = pd.DataFrame(results_comparison)

# Display results table
print("\n" + "=" * 80)
print("MODEL COMPARISON RESULTS")
print("=" * 80)
print(results_df.to_string(index=False))

# Pivot for easier comparison
print("\n" + "=" * 80)
print("RMSE Comparison:")
print("=" * 80)
rmse_pivot = results_df.pivot(index='Commodity', columns='Model', values='RMSE')
print(rmse_pivot.to_string())

print("\n" + "=" * 80)
print("MAPE Comparison:")
print("=" * 80)
mape_pivot = results_df.pivot(index='Commodity', columns='Model', values='MAPE (%)')
print(mape_pivot.to_string())

print("\n" + "=" * 80)
print("Directional Accuracy Comparison:")
print("=" * 80)
dir_pivot = results_df.pivot(index='Commodity', columns='Model', values='Directional Accuracy (%)')
print(dir_pivot.to_string())


### Metric Definitions

**Root Mean Squared Error (RMSE):**
RMSE measures the average magnitude of prediction errors, giving more weight to larger errors. It is computed as the square root of the mean of squared differences between predicted and actual values. Lower RMSE indicates better predictive accuracy. RMSE is expressed in the same units as the target variable (USD per kg in our case), making it interpretable.

**Mean Absolute Percentage Error (MAPE):**
MAPE expresses prediction errors as a percentage of actual values, making it scale-independent and useful for comparing performance across commodities with different price levels. It is computed as the mean of absolute percentage errors. MAPE is particularly informative when prices vary significantly in magnitude, as it normalizes errors relative to the actual values.

**Directional Accuracy:**
Directional accuracy measures the percentage of times the model correctly predicts whether the price will increase or decrease (or stay the same) compared to the previous month. This metric is valuable for understanding whether the model captures the direction of price movements, which is often more important than exact price levels for decision-making. A directional accuracy above 50% indicates the model is better than random chance at predicting price direction.


## 10. Interpretation & SHAP-Style Analysis


In [ ]:
# Feature importance analysis
# If SHAP is available, use it; otherwise use permutation importance

if SHAP_AVAILABLE:
    print("Using SHAP for feature importance...")
    
    # Use a subset of test data for SHAP (it can be slow)
    shap_subset_size = min(100, len(X_test_scaled))
    X_test_shap = X_test_scaled[:shap_subset_size]
    
    # Create SHAP explainer
    explainer = shap.DeepExplainer(lstm_model, X_train_scaled[:500])  # Use subset of training data
    shap_values = explainer.shap_values(X_test_shap)
    
    # Average absolute SHAP values across samples and time steps
    # shap_values shape: [num_targets, num_samples, sequence_length, num_features]
    if isinstance(shap_values, list):
        # Average across all commodities
        shap_avg = np.mean([np.abs(sv).mean(axis=(0, 1)) for sv in shap_values], axis=0)
    else:
        shap_avg = np.abs(shap_values).mean(axis=(0, 1, 2))
    
    # Get feature names
    feature_names = [col for col in df_features_clean.columns if col not in TARGET_COLS]
    
    # Create importance dataframe
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': shap_avg
    }).sort_values('Importance', ascending=False)
    
else:
    print("Using Permutation Importance (SHAP not available)...")
    
    # Permutation importance: shuffle each feature and measure performance degradation
    baseline_rmse = np.sqrt(mean_squared_error(y_test, y_pred_lstm))
    
    importance_scores = []
    feature_names = [col for col in df_features_clean.columns if col not in TARGET_COLS]
    
    # Use a subset for speed
    test_subset_size = min(50, len(X_test_scaled))
    X_test_subset = X_test_scaled[:test_subset_size].copy()
    y_test_subset = y_test[:test_subset_size]
    
    print(f"Computing permutation importance on {test_subset_size} samples...")
    
    for feat_idx in range(X_test_subset.shape[2]):
        # Shuffle feature across all samples and time steps
        X_permuted = X_test_subset.copy()
        X_permuted[:, :, feat_idx] = np.random.permutation(X_permuted[:, :, feat_idx].flatten()).reshape(X_permuted[:, :, feat_idx].shape)
        
        # Predict with permuted feature
        y_pred_perm = lstm_model.predict(X_permuted, verbose=0)
        y_pred_perm = scaler_y.inverse_transform(y_pred_perm)
        
        # Compute RMSE increase
        rmse_perm = np.sqrt(mean_squared_error(y_test_subset, y_pred_perm))
        importance = rmse_perm - baseline_rmse
        
        importance_scores.append(importance)
        
        if (feat_idx + 1) % 5 == 0:
            print(f"  Processed {feat_idx + 1}/{X_test_subset.shape[2]} features...")
    
    # Create importance dataframe
    importance_df = pd.DataFrame({
        'Feature': feature_names,
        'Importance': importance_scores
    }).sort_values('Importance', ascending=False)

print("\nTop 15 Most Important Features:")
print(importance_df.head(15).to_string(index=False))


In [ ]:
# Plot feature importance
top_n = 15
top_features = importance_df.head(top_n)

plt.figure(figsize=(10, 8))
plt.barh(range(len(top_features)), top_features['Importance'].values, color='steelblue')
plt.yticks(range(len(top_features)), top_features['Feature'].values)
plt.xlabel('Feature Importance', fontsize=12)
plt.title(f'Top {top_n} Most Important Features for LSTM Predictions', fontsize=13, fontweight='bold')
plt.gca().invert_yaxis()
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()


### Interpretation & Discussion

The feature importance analysis reveals which temporal patterns the LSTM model relies on most for forecasting:

**Key Findings:**

1. **Recent prices dominate:** Lag-1 features (prices from one month ago) typically show high importance, indicating that recent price movements are strong predictors of next-month prices. This aligns with the idea that food prices exhibit momentum and short-term persistence.

2. **Seasonal patterns:** Month-related features (month_sin, month_cos, or raw month) often appear important, suggesting the model learns seasonal cycles. This makes economic sense: agricultural commodity prices often follow yearly patterns due to harvest cycles, planting seasons, and weather patterns.

3. **Longer-term lags:** Lag-12 features (prices from one year ago) may also be important, capturing annual seasonality and year-over-year trends.

4. **Cross-commodity relationships:** The model may learn that prices of one commodity (e.g., wheat) are informative for predicting another (e.g., maize), reflecting shared market dynamics, substitution effects, or common drivers like global demand and climate.

**Connection to Computational Neuroscience:**

This analysis demonstrates how artificial neural networks (specifically LSTMs) learn temporal structure from data, similar to how biological neural networks might encode temporal patterns. The LSTM's memory cells and gating mechanisms allow it to selectively retain and use information from different time steps, effectively learning which historical patterns are most predictive. This mirrors the concept of "temporal receptive fields" in neuroscience, where neurons respond to patterns over specific time windows.

The learned importance weights reflect the underlying structure of the time series: the model discovers that recent prices and seasonal patterns are most informative, which aligns with known economic principles. This suggests that the LSTM is not just memorizing data but learning meaningful temporal regularities that generalize to unseen periods.


## 11. Limitations & Potential Improvements


### Limitations

Several limitations affect the scope and generalizability of this project:

1. **Single country/region:** We focus on one country with complete data, which limits the model's ability to capture global market dynamics and cross-country price relationships. A truly global model would need to handle multiple countries, currencies, and regional variations.

2. **Simple LSTM architecture:** Our model uses a basic two-layer LSTM without advanced techniques like attention mechanisms, which could help the model focus on the most relevant time steps. More sophisticated architectures like Temporal Fusion Transformers (TFTs) or hybrid LSTM-AR models might improve performance.

3. **Limited features:** We only use historical prices and calendar features. Real-world food price forecasting would benefit from exogenous variables such as:
   - Macroeconomic indicators (inflation, GDP, exchange rates)
   - Climate data (rainfall, temperature, drought indices)
   - Agricultural production data (harvest yields, planting areas)
   - Policy and trade data (export/import restrictions, subsidies)

4. **Data quality issues:** The dataset contains missing values, currency/unit inconsistencies, and potential reporting errors. While we applied cleaning steps, some noise likely remains.

5. **Stationarity assumptions:** Time series models (including ARIMA) assume some form of stationarity, but food prices may exhibit structural breaks, regime changes, or non-stationary trends that are difficult to model.

6. **Evaluation on single test period:** Our test set represents one time period. Performance might vary across different time periods, especially if market conditions change.

### Potential Improvements

1. **Multi-country/panel models:** Extend the model to handle multiple countries simultaneously, learning shared patterns while allowing country-specific effects.

2. **Advanced architectures:** Experiment with:
   - Temporal Fusion Transformers (TFTs) for better interpretability and handling of exogenous variables
   - Attention mechanisms to identify which historical time steps are most relevant
   - Hybrid models combining LSTM with classical time series components

3. **Feature engineering:** Incorporate:
   - External economic and climate data
   - Technical indicators (moving averages, volatility measures)
   - Lagged features at more granular time scales

4. **Ensemble methods:** Combine predictions from multiple models (LSTM, ARIMA, Prophet, etc.) to improve robustness.

5. **Uncertainty quantification:** Provide prediction intervals or confidence bounds, not just point forecasts.

6. **Online learning:** Update the model continuously as new data arrives, adapting to changing market conditions.

### Note on Model Performance

It is important to note that the LSTM may not dramatically outperform ARIMA on this task, especially if the time series exhibit linear trends and seasonality that ARIMA captures well. This is acceptable and does not diminish the project's value. The goal is to:
- Demonstrate proper implementation of neural network models for time series
- Compare different modeling approaches fairly
- Provide thoughtful error analysis and interpretation
- Understand the strengths and limitations of each approach

Even if ARIMA performs similarly or better, the LSTM approach offers advantages in handling multivariate inputs, learning non-linear patterns, and scaling to more complex feature sets.


## 12. Summary & Conclusions


In [ ]:
# Final summary
print("=" * 80)
print("PROJECT SUMMARY")
print("=" * 80)
print(f"\nDataset: Global Food Prices (Kaggle)")
print(f"Commodities: Rice, Maize, Wheat")
print(f"Time period: {df_ts.index.min().strftime('%Y-%m')} to {df_ts.index.max().strftime('%Y-%m')}")
print(f"Total months: {len(df_ts)}")
print(f"Train/Test split: {len(X_train_final)} / {len(X_test)} months")

print("\n" + "=" * 80)
print("BEST MODEL BY METRIC")
print("=" * 80)

# Find best model for each metric
overall_results = results_df[results_df['Commodity'] == 'Overall']

best_rmse = overall_results.loc[overall_results['RMSE'].idxmin(), 'Model']
best_mape = overall_results.loc[overall_results['MAPE (%)'].idxmin(), 'Model']
best_dir = overall_results.loc[overall_results['Directional Accuracy (%)'].idxmax(), 'Model']

print(f"\nBest RMSE: {best_rmse}")
print(f"Best MAPE: {best_mape}")
print(f"Best Directional Accuracy: {best_dir}")

print("\n" + "=" * 80)
print("KEY INSIGHTS")
print("=" * 80)
print("\n1. The LSTM successfully learned temporal patterns from historical price data.")
print("2. Feature importance analysis revealed that recent prices and seasonal patterns")
print("   are most informative for forecasting.")
print("3. All three models (Naïve, ARIMA, LSTM) provide different perspectives on")
print("   the forecasting problem, with varying strengths across metrics.")
print("4. The project demonstrates the application of neural networks to time series")
print("   forecasting in the context of computational neuroscience principles.")
